# Data Aggregation

The process where information is gathered and presented in a summary form with intent to prepare data for statiatical analysis.


## Task

To aggregate information between the [exam schedule](http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2019.html) and the [class schedule](http://atlas.college.emory.edu/class-schedules/spring-2019.php).

# 1. Exam Schedule

## 1.1. HTML Parsing

Retrieve the HTML source from the exam schedule page:

In [1]:
import requests

url = 'http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2019.html'
r = requests.get(url)
print(r.text[:82])  # print only the first line

<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">


Find the table containing the exam schedule information from the source:

<img src="res/exam-schedule-spring-2019.png">

```html
<table cellpadding="0" cellspacing="0" class="table table-striped">
<thead>
<tr>
  <td>Class Meeting Time</td>
  <td>Exam Day</td>
  <td>Exam Date</td>
  <td>Exam Time</td>
</tr>
</thead>
<tbody>
<tr>
  <td>08:00 MWF</td>
  <td>Friday</td>
  <td>3-May</td>
  <td>11:30 A.M - 2:00 P.M</td>
</tr>
<tr>
  <td>08:00 TThF</td>
  <td>Friday</td>
  <td>3-May</td>
  <td>3:00 P.M - 5:30 P.M</td>
</tr>
```

Retrieve the exam schedule information from the table:

In [2]:
from bs4 import BeautifulSoup

html = BeautifulSoup(r.text, 'html.parser')
tbody = html.find('tbody')
schedule = []

for tr in tbody.find_all('tr'):
    tds = tr.find_all('td')
    class_time = tds[0].string.strip()
    exam_day   = tds[1].string.strip()
    exam_date  = tds[2].string.strip()
    exam_time  = tds[3].string.strip()
    schedule.append([class_time, exam_day, exam_date, exam_time])

print(schedule[0])
print(schedule[1])

['08:00 MWF', 'Friday', '3-May', '11:30 A.M - 2:00 P.M']
['08:00 TThF', 'Friday', '3-May', '3:00 P.M - 5:30 P.M']


## 1.2. Regular Expressions

Split each class meeting time into (hour, minute, days):

In [3]:
import re

TIME_DAYS = re.compile('(\d{1,2}):(\d\d)\s+([A-Za-z]+)')

m = TIME_DAYS.match('8:00 MW')
print('Hour: %2s, Minute: %2s, Day(s): %s' % (m.group(1), m.group(2), m.group(3)))

m = TIME_DAYS.match('12:30 TThF')
print('Hour: %2s, Minute: %2s, Day(s): %s' % (m.group(1), m.group(2), m.group(3)))

Hour:  8, Minute: 00, Day(s): MW
Hour: 12, Minute: 30, Day(s): TThF


If the input string does not match the expression, `None` is returned:

In [4]:
m = TIME_DAYS.match('Math*')
print(m)

None


### Exercise

Write a regular expression that handles various ways to indicate time:

```python
['08:00', '12:30', '2:30pm', '2:30 pm', '2:30PM', '2:30P.M', '2:30P.M.', '2:30 PM.']
```

In [5]:
times = ['08:00', '12:30', '2:30pm', '2:30 pm', '2:30PM', '2:30P.M', '2:30P.M.', '2:30 PM.']
TIME = re.compile('(\d{1,2}):(\d\d)\s*([AaPp]\.?\s*[Mm]\.?)?')

for time in times:
    m = TIME.match(time)
    hour   = m.group(1)
    minute = m.group(2)
    period = m.group(3)
    print('%10s : (%2s, %2s, %s)' % (time, hour, minute, period))

     08:00 : (08, 00, None)
     12:30 : (12, 30, None)
    2:30pm : ( 2, 30, pm)
   2:30 pm : ( 2, 30, pm)
    2:30PM : ( 2, 30, PM)
   2:30P.M : ( 2, 30, P.M)
  2:30P.M. : ( 2, 30, P.M.)
  2:30 PM. : ( 2, 30, PM.)


## 1.3. Normalization

Normalize the formats of time and days.

### Exercise

Write a function that converts the above matched results to military time (e.g., `"3:30 P.M"` &rarr; `1530`):

```python
def norm_time(hour: str, minute: str, period: str) -> int:
    # TODO: to be updated
    return 0
```

In [6]:
from typing import Optional

def norm_time(hour: str, minute: str, period: Optional[str]=None) -> int:
    h = int(hour)
    m = int(minute)

    if period and period[0].upper() == 'P':
        h += 12

    return h * 100 + m

In [7]:
for time in times:
    m = TIME.match(time)
    n = norm_time(m.group(1), m.group(2), m.group(3))
    print('%10s : %4d' % (time, n))

     08:00 :  800
     12:30 : 1230
    2:30pm : 1430
   2:30 pm : 1430
    2:30PM : 1430
   2:30P.M : 1430
  2:30P.M. : 1430
  2:30 PM. : 1430


### Exercise

Write a function that coverts days into a binary form, then converts the binary form into an integer (e.g., `"MWF"` &rarr; `"10101"` &rarr; `21`): 

```python
def norm_days(days: str) -> int:
    # TODO: to be updated
    return 0
```

In [8]:
def norm_days(days: str) -> int:
    DAYS = [('M', 0), ('TU', 1), ('W', 2), ('TH', 3), ('F', 4)]
    days = days.upper()
    b = ['0'] * 5

    for d, i in DAYS:
        if d in days:
            b[i] = '1'
            days = days.replace(d, '')
    
    if 'T' in days:
        b[1] = '1'
        days = days.replace('T', '')

    return int(''.join(b), 2)

In [9]:
days = ['MWF', 'TuTh', 'MTuWThF', 'TThF', 'MWFf']

for day in days:
    n = norm_days(day)
    print('%7s %5s %2d' % (day, bin(n)[2:], n))

    MWF 10101 21
   TuTh  1010 10
MTuWThF 11111 31
   TThF  1011 11
   MWFf 10101 21


## 1.4. Putting Together

Write a function that takes the exam schedule URL and returns a dictionary where the key is the normalized class meeting time and the value is its exam schedule information.

In [10]:
from typing import Dict, Tuple

def extract_exam_schedule(url) -> Dict[Tuple[int, int], Tuple[str, str, str]]:
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html.parser')
    tbody = html.find('tbody')
    schedule = {}

    for tr in tbody.find_all('tr'):
        tds = tr.find_all('td')
        class_time = tds[0].string.strip()
        m = TIME_DAYS.match(class_time)
        if m:
            time = norm_time(int(m.group(1)), int(m.group(2)))
            days = norm_days(m.group(3))
            key  = (time, days)
            exam_day  = tds[1].string.strip()
            exam_date = tds[2].string.strip()
            exam_time = tds[3].string.strip()
            schedule[key] = (exam_day, exam_date, exam_time)

    return schedule

In [11]:
exam_schedule = extract_exam_schedule(url)
for k, v in exam_schedule.items():
    print('%14s : %s' % (k, v))

     (800, 21) : ('Friday', '3-May', '11:30 A.M - 2:00 P.M')
     (800, 11) : ('Friday', '3-May', '3:00 P.M - 5:30 P.M')
     (830, 20) : ('Friday', '3-May', '11:30 A.M - 2:00 P.M')
     (830, 10) : ('Friday', '3-May', '3:00 P.M - 5:30 P.M')
     (900, 21) : ('Friday', '3-May', '11:30 A.M - 2:00 P.M')
     (900, 11) : ('Friday', '3-May', '3:00 P.M - 5:30 P.M')
    (1000, 20) : ('Thursday', '2-May', '8:00 A.M - 10:30 A.M')
    (1000, 21) : ('Thursday', '2-May', '8:00 A.M - 10:30 A.M')
    (1000, 10) : ('Friday', '3-May', '8:00 A.M - 10:30 A.M')
    (1000, 11) : ('Friday', '3-May', '8:00 A.M - 10:30 A.M')
    (1100, 21) : ('Wednesday', '8-May', '8:00 A.M - 10:30 A.M')
    (1100, 11) : ('Tuesday', '7-May', '8:00 A.M - 10:30 A.M')
    (1130, 20) : ('Wednesday', '8-May', '8:00 A.M - 10:30 A.M')
    (1130, 10) : ('Tuesday', '7-May', '8:00 A.M - 10:30 A.M')
    (1200, 21) : ('Wednesday', '8-May', '3:00 P.M - 5:30 P.M')
    (1200, 11) : ('Wednesday', '8-May', '3:00 P.M - 5:30 P.M')
     (100, 

### Question

* Which exam schedules are not extracted by the `extract_exam_schedule` function, and why?
* How to modify the `extract_exam_schedule` function to handle times for afternoon classes correctly?

# 2. Class Schedule

## 2.1. HTML Parsing

Retrieve the HTML source from the class schedule page:

In [12]:
url = 'http://atlas.college.emory.edu/class-schedules/spring-2019.php'
r = requests.get(url)
print(r.text[:53])

<html xmlns="http://www.w3.org/1999/xhtml" lang="en">


Extract the class schedule information from the source:

<img src="res/class-schedule-spring-2019.png">

```html
<tr>
  <td>AAS</td>
  <td>100</td>
  <td>1</td>
  <td>HSC</td>
  <td>AAS 100 - 1</td>
  <td>
    <div class="titles">
      <div class="course">Intro To Afric Amer Studies</div>
      <div class="class"></div>
    </div>
  </td>
  <td>1416</td>
  <td>4</td>
  <td>HSC</td>
  <td>
    <table class="table-schedule">
      <tr class="schedule">
        <td class="day">TuTh</td>
        <td class="time">2:30 - 3:45<small>pm</small></td>
        <td class="location">Rich Building 104</td>
        <td class="instructor"><div class="name">Gordon, Michelle</div></td>
      </tr>
      <tr class="schedule">
        <td class="day">M</td>
        <td class="time">5:00 - 7:00<small>pm</small></td>
        <td class="location">Callaway Center S103</td>
        <td class="instructor"><div class="name">Gordon, Michelle</div></td>
      </tr>
    </table>
  </td>
  <td>This course introduces students to the multiple disciplines ...</td>
</tr>
```

### Exercise

Write a code that extracts the following information from the HTML source:

* Program of the course (e.g., AAS)
* Course number (e.g., 100)
* Course section (e.g., 1)
* Course title (e.g., Introduction to African American Studies)
* Class number in OPUS (e.g., 1416).
* Class days (TuTh).
* Class time (2:30 - 3:45pm).
* Instructor (Gordon, Michelle).

In [13]:
html = BeautifulSoup(r.text, 'html.parser')
schedule = []

for tr in html.find_all('tr'):
    td1 = tr.find_all('td')
    if len(td1) < 10: continue
        
    program = td1[0].string.strip()
    number = td1[1].string.strip()
    section = td1[2].string.strip()
    title = td1[5].text
    opus = int(td1[6].string)

    td2 = td1[9].find_all('td')
    days = td2[0].string
    if days is None: continue
    else: days = days.strip()
    time = td2[1].text.strip()
    instructor = td2[3].string
    
    schedule.append((program, number, section, title, opus, days, time, instructor))

print(schedule[0])

('AAS', '100', '1', 'Intro To Afric Amer Studies', 1416, 'TuTh', '2:30 - 3:45pm', 'Gordon, Michelle')


## 2.2. Normalization

Normalize the formats of time and days.

### Exercise

Write a code that creates a dictionary where the key is a tuple of normalized (time, days) and the value is the count of classes during that period.

In [14]:
TIME = re.compile('(\d{1,2}):(\d\d)')
course_count = {}

for s in schedule:
    days = norm_days(s[5])
    m = TIME.match(s[6])
    time = norm_time(m.group(1), m.group(2))
    key = (time, days)
    course_count[key] = course_count.get(key, 0) + 1

# t[0]: tuple of (time, days), t[1]: count
course_count = sorted(course_count.items(), key=lambda t: t[1], reverse=True)
for k, v in course_count[:10]:
    print('Time: %4d, Days: %2d, Count: %3d' % (*k, v))

Time: 1000, Days: 10, Count: 100
Time:  100, Days: 10, Count:  88
Time: 1130, Days: 10, Count:  79
Time: 1000, Days: 20, Count:  77
Time:  230, Days: 10, Count:  70
Time:  100, Days: 20, Count:  67
Time: 1130, Days: 20, Count:  67
Time:  230, Days: 20, Count:  60
Time:  400, Days: 10, Count:  53
Time:  400, Days: 20, Count:  32


### Question

* How to modify the above code to handle times for afternoon classes correctly?

## 2.3. Putting Together

Write a function that takes the class schedule URL and returns a dictionary where the key is the class ID in OPUS and the value is a tuple of the followings:

* Program of the course
* Course number
* Course section
* Course title
* Class number in OPUS
* Class days
* Class time
* Instructor

In [15]:
def extract_class_schedule(url) -> Dict[int, Tuple[str, str, str, str, int, str, str, str]]:
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html.parser')
    schedule = {}

    for tr in html.find_all('tr'):
        td1 = tr.find_all('td')
        if len(td1) < 10: continue

        program = td1[0].string.strip()
        number = td1[1].string.strip()
        section = td1[2].string.strip()
        title = td1[5].text.strip()
        opus = int(td1[6].string)

        td2 = td1[9].find_all('td')
        days = td2[0].string
        if days is None: continue
        else: days = days.strip()
        time = td2[1].text.strip()
        instructor = td2[3].string
        
        schedule[opus] = (program, number, section, title, opus, days, time, instructor)

    return schedule

In [16]:
schedule = extract_class_schedule(url)

In [17]:
print(schedule[1416])

('AAS', '100', '1', 'Intro To Afric Amer Studies', 1416, 'TuTh', '2:30 - 3:45pm', 'Gordon, Michelle')


# 3. Aggregation

## 3.1. Information Retrieval

Retrieve the exam schedule by the class number in OPUS.

In [18]:
url_exam = 'http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2019.html'
url_class = 'http://atlas.college.emory.edu/class-schedules/spring-2019.php'

exam_schedule = extract_exam_schedule(url_exam)
class_schedule = extract_class_schedule(url_class)

### Exercise

Write a function that takes `exam_schedule`, `class_schedule`, and an OPUS class number, and returns the exam schedule of the corresponding class if available; otherwise, `None`:

```python
def get_exam_schedule(
    exam_schedule: Dict[Tuple[int, int], Tuple[str, str, str]], 
    class_schedule: Dict[int, Tuple[str, str, str, str, int, str, str, str]], 
    opus: int) -> Tuple[str, str, str]:
    # TODO: to be updated
    return None
```

In [19]:
def get_exam_schedule(opus: int, exam_schedule: Dict[Tuple[int, int], Tuple[str, str, str]], class_schedule: Dict[int, Tuple[str, str, str, str, int, str, str, str]]) -> Tuple[str, str, str]:
    s = class_schedule.get(opus, None)
    if s is None: return None
    days = norm_days(s[5])
    m = TIME.match(s[6])
    time = norm_time(m.group(1), m.group(2))
    key = (time, days)
    return exam_schedule.get(key, None)

In [20]:
get_exam_schedule(1416, exam_schedule, class_schedule)

('Thursday', '2-May', '3:00 P.M - 5:30 P.M')

## 3.2. HTML Generation

Generate an HTML page that comprises the aggregated information.

In [21]:
def create_html(html_file: str, program: str, exam_schedule: Dict[Tuple[int, int], Tuple[str, str, str]], class_schedule: Dict[int, Tuple[str, str, str, str, int, str, str, str]]):
    soup = BeautifulSoup("<html></html>", 'html.parser')
    html = soup.html

    table = soup.new_tag('table')
    html.append(table)

    thead = soup.new_tag('thead')
    table.append(thead)

    tr = soup.new_tag('tr')
    thead.append(tr)

    for s in ['Couse', 'Opus', 'Title', 'Instructor', 'Exam Schedule']:
        td = soup.new_tag('td')
        td.string = s
        tr.append(td)

    tbody = soup.new_tag('tbody')
    table.append(tbody)

    for k, v in class_schedule.items():
        if v[0] != program: continue
        course = v[0] + v[1] + '-' + v[2]
        opus = str(v[4])
        title = v[3]
        instructor = str(v[-1])
        t = get_exam_schedule(k, exam_schedule, class_schedule)
        if t is None: continue
        ex_schedule = ', '.join(list(t))

        tr = soup.new_tag('tr')
        tbody.append(tr)

        for s in [course, opus, title, instructor, ex_schedule]:
            td = soup.new_tag('td')
            td.string = s
            tr.append(td)

    with open(html_file, 'w') as fout:
        fout.write(soup.prettify())

In [22]:
create_html('res/qtm.html', 'QTM', exam_schedule, class_schedule)

In [23]:
s = set([v[0] for k, v in class_schedule.items()])
fout = open('qtm.html', 'w')

for q in sorted(list(s)):
    fout.write('<a href="%s.html">%s</a><br>\n' % (q.lower(), q))